# Summary History

In [17]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["LANGSMITH_PROJECT"]

'LANGCHAIN-BASIC'

In [18]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0.1,
    model='gpt-4.1-mini',
    verbose=True
)

In [19]:
from typing import Dict, Tuple
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables.utils import ConfigurableFieldSpec

In [20]:
system_prompt = """
추석 덕담을 나누는 상황이야 답변을 해줘
"""

In [5]:
stores : Dict[Tuple[str, str], InMemoryChatMessageHistory] = {}

def get_session_history(session_id: str, conversation_id: str) -> BaseChatMessageHistory:
    
    key = (session_id, conversation_id)
    
    if key not in stores:
        stores[key] = InMemoryChatMessageHistory()
        
    return stores[key]

In [6]:
# 대화내용 요약 chain
summaries : Dict[Tuple[str, str], str] = {}

summaries_prompt = ChatPromptTemplate.from_messages([
    '다음 대화 내용을 5단어로 요약해. 불필요한 내용은 제외!\n 대화내용 : {content_text}'   
])

summaries_chain = summaries_prompt | model | StrOutputParser()

# threshold만큼 요약
def maybe_summarize(session_id: str, conversation_id: str, threshold: int=8):
    
    store = get_session_history(session_id, conversation_id)
    
    if len(store.messages) > threshold:
        
        content_text = "" # 지금까지 대화내용을 합친 글자
        for i in store.messages:
            content_text += i.content + '\n'
            
        summaries[(session_id, conversation_id)] = summaries_chain.invoke({'content_text' : content_text})

In [7]:
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_prompt), # system prompt가 여러개면 하나로 합쳐서 적용됨
    ('system', '과거 요약:\n{summary}'),
    MessagesPlaceholder(variable_name='history'),
    ('user', '{question}')
])

chain = prompt_template | model | StrOutputParser()
chain

ChatPromptTemplate(input_variables=['history', 'question', 'summary'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotat

In [8]:
with_summary_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='question',
    history_messages_key='history',
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ]
)

In [ ]:
def ask(question: str, session_id: str, conversation_id: str) -> str:
    
    maybe_summarize(session_id, conversation_id)
    config = {'configurable': {'session_id': session_id, 'conversation_id': conversation_id}}
    
    return with_summary_chain.invoke(
        {'question': question, 
        'summary': summaries.get((session_id, conversation_id), 'empty')},
        config
        )

In [ ]:
ask("오래오래 건강하세요", "walker0625", "conv-1")

감사합니다! 가족 모두 건강하고 행복한 한가위 보내시길 바랍니다. 풍성한 명절 되세요!


In [ ]:
ask("하는 일 다 잘되렴", "walker0625", "conv-1")

고마워요! 당신도 하시는 모든 일마다 좋은 결과 있길 진심으로 응원할게요. 행복한 추석 보내세요!


In [ ]:
ask("새해에는 돈 많이 벌으렴", "walker0625", "conv-1")

감사합니다! 새해에는 부자 되시고, 원하는 모든 일 이루시길 바랍니다. 즐거운 추석 보내세요!


In [ ]:
ask("새해엔 나쁜 일이 없길바래", "walker0625", "conv-1")

정말 따뜻한 덕담이에요! 새해에는 좋은 일만 가득하고, 건강과 행복이 늘 함께하길 진심으로 기원합니다. 행복한 추석 보내세요!


In [15]:
summaries.get(("walker0625", "conv-1"))

In [16]:
stores[("walker0625", "conv-1")].messages

[HumanMessage(content='오래오래 건강하세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='감사합니다! 가족 모두 건강하고 행복한 한가위 보내시길 바랍니다. 풍성한 명절 되세요!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='하는 일 다 잘되렴', additional_kwargs={}, response_metadata={}),
 AIMessage(content='고마워요! 당신도 하시는 모든 일마다 좋은 결과 있길 진심으로 응원할게요. 행복한 추석 보내세요!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='새해에는 돈 많이 벌으렴', additional_kwargs={}, response_metadata={}),
 AIMessage(content='감사합니다! 새해에는 부자 되시고, 원하는 모든 일 이루시길 바랍니다. 즐거운 추석 보내세요!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='새해엔 나쁜 일이 없길바래', additional_kwargs={}, response_metadata={}),
 AIMessage(content='정말 따뜻한 덕담이에요! 새해에는 좋은 일만 가득하고, 건강과 행복이 늘 함께하길 진심으로 기원합니다. 행복한 추석 보내세요!', additional_kwargs={}, response_metadata={})]